In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    #return f"Email sent"
    return f"Email sent with body:\n{body}"

In [14]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model="gpt-5-nano",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [15]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [5]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'Thanks '
                                                                          'for '
                                                                          'the '
                                                                          'heads '
                                                                          'up. '
                                                                          'No '
                                                                          'problem—I’m '
                                                                          'flexible.\n'
                                                                          '\n'
           

In [6]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': 'Hi John,\n\nThanks for the heads up. No problem—I’m flexible.\n\nWould either 2:00 PM or 4:00 PM tomorrow work for you? If not, please suggest a couple of alternative times and I’ll adjust.\n\nIf you’d prefer, we can switch to a quick call instead of a meeting.\n\nBest regards,\nSeán'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'body': 'Hi John,\\n\\nThanks for the heads up. No problem—I’m flexible.\\n\\nWould either 2:00 PM or 4:00 PM tomorrow work for you? If not, please suggest a couple of alternative times and I’ll adjust.\\n\\nIf you’d prefer, we can switch to a quick call instead of a meeting.\\n\\nBest regards,\\nSeán'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='d3ccd8800de0ec524f1a026427d5afae')]


In [7]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John,

Thanks for the heads up. No problem—I’m flexible.

Would either 2:00 PM or 4:00 PM tomorrow work for you? If not, please suggest a couple of alternative times and I’ll adjust.

If you’d prefer, we can switch to a quick call instead of a meeting.

Best regards,
Seán


## Approve

In [8]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='fe0c45fd-dba6-44b9-a8dd-9d36e97ac506'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 403, 'prompt_tokens': 157, 'total_tokens': 560, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqdtbdAhze5XNbes1tlfvuQoEEuvT', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5549-aef6-7591-8999-ab2364d3bd36-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'call_5i

## Reject

In [9]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='fe0c45fd-dba6-44b9-a8dd-9d36e97ac506'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 403, 'prompt_tokens': 157, 'total_tokens': 560, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqdtbdAhze5XNbes1tlfvuQoEEuvT', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5549-aef6-7591-8999-ab2364d3bd36-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'call_5i

In [17]:
#print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

## Edit

In [16]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='aa2890c3-ae39-402a-8c67-178d6d6c908a'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 659, 'prompt_tokens': 157, 'total_tokens': 816, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqdyHf4QoCC6hsE5HPTIwnLJxknuL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b554e-19c0-7363-ab77-2e6c1cb2bb01-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'call_EX